In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./data/FAOSTAT_data_1961_2019.csv')

#drop numeric codes that do not apply
cleaned_df = df.drop(['Area Code', 'Item Code', 'Source Code', 'Source', 'Element Code'], axis=1)

#extract columns belonging to the USA 
cleaned_df = cleaned_df[cleaned_df['Area'] == 'United States of America']

#rename Items column to Activity
cleaned_df.rename(columns={'Item':'Activity'}, inplace=True)

#reset the index 
cleaned_df = cleaned_df.reset_index(drop=True)

cleaned_df.head()


,Area,Activity,Element,Unit,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,United States of America,Enteric Fermentation,Emissions (CH4),kilotonnes,6876.7867,6980.14,7136.368,7251.5884,7253.3471,7171.6855,...,6008.5861,5954.5384,5865.1366,5805.0675,5728.0324,5766.8023,5920.9829,6020.3071,6064.9878,6092.2606
1,United States of America,Enteric Fermentation,Emissions (CH4),kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,...,6870.1741,6773.3172,6685.8465,6635.4056,6582.8275,6675.4774,6889.6582,7032.3003,7119.0725,7142.3477
2,United States of America,Enteric Fermentation,Emissions (CO2eq) from CH4 (AR5),kilotonnes,192550.0266,195443.92,199818.304,203044.4759,203093.7178,200807.1943,...,168240.4108,166727.0752,164223.8248,162541.8900,160384.9086,161470.4631,165787.5208,168568.5991,169819.6590,170583.2966
3,United States of America,Enteric Fermentation,Emissions (CO2eq) from CH4 (AR5),kilotonnes,NaN,NaN,NaN,NaN,NaN,NaN,...,192364.8745,189652.8824,187203.7009,185791.3572,184319.1694,186913.3660,192910.4293,196904.4088,199334.0288,199985.7362
4,United States of America,Enteric Fermentation,Emissions (CO2eq) (AR5),kilotonnes,192550.0266,195443.92,199818.304,203044.4759,203093.7178,200807.1943,...,168240.4108,166727.0752,164223.8248,162541.8900,160384.9086,161470.4631,165787.5208,168568.5991,169819.6590,170583.2966


In [28]:
#create an emission activity and emission type list. Each value is a tuple, 0th -> activity, 1st -> type of emission
emission_activity_and_type = list(cleaned_df.groupby(['Activity','Element']).indices.keys())


'''Anthropogenic climate change is caused by multiple climate pollutants, with CO2, CH4, and N2O the three largest individual contributors to global warming. Agriculture and food production is associated with all three of these gases, but direct agricultural emissions are unusual in being dominated by CH4 and N2O.'''

#filter the emissions list to only contain the biggest contributers to argiculture
CO2_gases = list(filter(lambda emission: 'CO2' in emission[1],emission_activity_and_type))
CH4_gases = list(filter(lambda emission: 'CH4' in emission[1],emission_activity_and_type))
N2O_gases = list(filter(lambda emission: 'N2O' in emission[1],emission_activity_and_type))

#Because Area and Unit are the same value in every row (USA, Kilotonnes respectively), I will drop those. Set the column indicies to Activity
CO2_df = cleaned_df[cleaned_df['Element'].isin([el[1] for el in CO2_gases])].drop(['Area', 'Unit'], axis=1).reset_index(drop=True)
CH4_df = cleaned_df[cleaned_df['Element'].isin([el[1] for el in CH4_gases])].drop(['Area', 'Unit'], axis=1).reset_index(drop=True)
N2O_df = cleaned_df[cleaned_df['Element'].isin([el[1] for el in N2O_gases])].drop(['Area', 'Unit'], axis=1).reset_index(drop=True)


#Clean up the emissions dataframe consisting of only emission activity/element and year
def create_emissions_frame(columns, df):
    df['Emission'] = df[columns[0]].astype(str) + '_' + df[columns[1]]
    df.drop(columns, axis=1, inplace=True)
    df.insert(0,'Emission', df.pop('Emission'))
    df = df.set_index('Emission').transpose().fillna(0).reset_index()
    df['index'] = df['index'].str.lstrip('Y')
    return df.rename(columns={'index':'Year'}).set_index('Year')
    


CO2_df = create_emissions_frame(['Activity', 'Element'], CO2_df)
CH4_df = create_emissions_frame(['Activity', 'Element'], CH4_df)
N2O_df = create_emissions_frame(['Activity', 'Element'], N2O_df)

Manure Management_Emissions (N2O)
Manure Management_Emissions (N2O)
Manure Management_Emissions (CO2eq) from N2O (AR5)
Manure Management_Emissions (CO2eq) from N2O (AR5)
Synthetic Fertilizers_Direct emissions (N2O)
Synthetic Fertilizers_Direct emissions (N2O)
Synthetic Fertilizers_Indirect emissions (N2O)
Synthetic Fertilizers_Emissions (N2O)
Synthetic Fertilizers_Emissions (N2O)
Synthetic Fertilizers_Emissions (CO2eq) from N2O (AR5)
Synthetic Fertilizers_Emissions (CO2eq) from N2O (AR5)
Manure applied to Soils_Direct emissions (N2O)
Manure applied to Soils_Direct emissions (N2O)
Manure applied to Soils_Indirect emissions (N2O)
Manure applied to Soils_Emissions (N2O)
Manure applied to Soils_Emissions (N2O)
Manure applied to Soils_Emissions (CO2eq) from N2O (AR5)
Manure applied to Soils_Emissions (CO2eq) from N2O (AR5)
Manure left on Pasture_Direct emissions (N2O)
Manure left on Pasture_Direct emissions (N2O)
Manure left on Pasture_Indirect emissions (N2O)
Manure left on Pasture_Emissio

[]

In [4]:
#visualize our data
def visualize_emissions(df):
    year = df.index.values
    

array(['1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'], dtype=object)